<a href="https://colab.research.google.com/github/kayeschong/breaking2048/blob/master/Greedy_Score_Based_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import gym
from gym import spaces


class Env2048(gym.Env):
    LEFT = 0
    UP = 1
    RIGHT = 2
    DOWN = 3

    def __init__(self, height=4, width=4):
        super().__init__()

        self.height = height
        self.width = width
        self.n_tiles = self.height * self.width

        #self.action_space = spaces.Discrete(4)
        self.action_space = np.array([0,1,2,3])
        self.action_map = {0:"left", 1:"up", 2:"right", 3:"down"}
        self.observation_space = spaces.Box(0, 2 ** self.n_tiles, shape=(self.height, self.width), dtype=np.int)
        self.reset()
    
    def clone(self):
        grid_copy = Env2048()
        grid_copy.grid = np.copy(self.grid)
        return grid_copy

    def step(self, action):
        # Execute one time step within the environment
        """
        0 = left
        1 = up
        2 = right
        3 = down
        """
        assert action in self.action_space, f"Invalid action {action} | {self.action_space}"
        self.grid, moved = self._move(self.grid, action)

        obs = self.grid.copy()
        if moved:
            reward = 1
        else:
            reward = -1
        done = self._game_lost(self.grid)
        info = {
            "has_moved": moved,
            "last_action": action,
        }
        return obs, reward, done, info

    def reset(self):
        # Reset the state of the environment to an initial state
        self.grid = np.zeros(shape=(4, 4), dtype=np.int)

        self.grid = self.generate_tile(self.grid)
        self.grid = self.generate_tile(self.grid)
        return self.grid.copy()

    def render(self, mode='human'):
        # Render the environment to the screen
        print(self.grid)

    def get_highest_tile(self):
        return np.amax(self.grid)
    
    def get_valid_moves(self):
        '''
        Output:
            list[int]: Valid moves for current grid
        '''
        return np.array([move for move in range(4) if self.is_movable(move)])
    
    def get_invalid_moves(self):
        '''
        Output:
            list[int]: Invalid moves for current grid
        '''
        return np.array([move for move in range(4) if not self.is_movable(move)])
    
    def get_empty_cells(self):
        '''
        Output:
            list[tuples]: Positions of empty cells
        '''
        cells = []
        for x in range(4):
            for y in range(4):
                if self.grid[x][y] == 0:
                    cells.append((x,y))
        return cells

    def generate_tile(self, grid):
        '''
        Output:
            numpy array: New grid with randomly generated tile
        '''
        next_grid = np.copy(grid)
        if self._num_empty_tiles(next_grid) > 0:
            empty_positions = np.argwhere(next_grid.flatten() == 0).flatten()
            position_choice = np.random.choice(empty_positions)
            position_2d = np.unravel_index(position_choice, next_grid.shape)

            value_choice = np.random.choice([2, 4], p=[0.9, 0.1])

            next_grid[position_2d] = value_choice
        return next_grid

    def insert_tile(self, position, value):
        '''
        insert tile into position in grid
        '''
        self.grid[position[0]][position[1]] = value

    def is_movable(self, direction):
        '''
        Output:
            bool: If there is a valid move for current grid
        '''
        _, moved = self._move(self.grid, direction)
        return moved

    def _left(self, grid):
        next_grid = grid.copy()
        next_grid, flushed = self._flush_left(next_grid)
        next_grid, joined = self._join_left(next_grid)
        # Flush again if joined
        if joined:
            next_grid, _ = self._flush_left(next_grid)

        moved = flushed or joined
        if moved:
            next_grid = self.generate_tile(next_grid)
        return next_grid, moved

    def _move(self, grid, direction: int):
        next_grid = grid.copy()
        next_grid = np.rot90(next_grid, direction)
        next_grid, moved = self._left(next_grid)
        next_grid = np.rot90(next_grid, -direction)

        return next_grid, moved

    def _flush_left(self, grid):
        flushed_grid = np.zeros_like(grid)
        for i, row in enumerate(grid):
            non_zero = row[row != 0]
            n_non_zero = len(non_zero)
            flushed_grid[i, :n_non_zero] = non_zero

        flushed = (flushed_grid != grid).any()
        return flushed_grid, flushed

    def _join_left(self, grid):
        joined_grid = grid.copy()
        for col_index in range(joined_grid.shape[1] - 1):
            curr_col = joined_grid[:, col_index]
            next_col = joined_grid[:, col_index + 1]
            joinable = (curr_col == next_col) & (next_col != 0)
            curr_col[joinable] *= 2
            next_col[joinable] = 0

        joined = (grid != joined_grid).any()
        return joined_grid, joined

    def _num_empty_tiles(self, grid):
        '''
        Output:
            int: Number of empty tiles on group
        '''
        return (grid == 0).sum()

    def _game_lost(self, grid):
        '''
        Output:
            bool: If no more valid moves
        '''
        lost = all([not self.is_movable(direction) for direction in range(4)])
        return lost

    def __str__(self):
        return np.array2string(self.grid)
    
    def get_random_valid_move(self):
        return np.random.choice(self.get_valid_moves())  

    def random_play_simulation_run(self):
        test_env = self.clone()
        done = False

        while not done:
            try:
                action = test_env.get_random_valid_move()
                observation, reward, done, info = test_env.step(action)

            except:
                done = True
                
            if done:
                break

        return test_env.get_highest_tile()

In [ ]:
# Greedy Algorithm

env = Env2048()
env.reset()

done = False
i = 0

while not done:
    print("Time Step: {}".format(i))
    env.render()
    print(env.get_valid_moves())

    move_value = np.zeros(4)

    for move in env.get_valid_moves():
        test_env = env.clone()
        observation, reward, done, info = test_env.step(move)
        for run in range(1,11):
            move_value[move] += (test_env.random_play_simulation_run() - move_value[move])/run
    
    print(move_value)
    action = np.argmax(move_value)

    print(env.action_map[action])
    observation, reward, done, info = env.step(action)
    i += 1
    if done:
        print(env.get_highest_tile())
        break  
    
env.close()

Time Step: 0
[[0 0 0 0]
 [0 2 0 0]
 [0 0 0 0]
 [0 0 0 2]]
[0 1 2 3]
[ 86.4  92.8 115.2 105.6]
right
Time Step: 1
[[0 0 0 2]
 [0 0 0 2]
 [0 0 0 0]
 [0 0 0 2]]
[0 1 3]
[105.6  80.    0.   83.2]
left
Time Step: 2
[[2 0 0 0]
 [2 2 0 0]
 [0 0 0 0]
 [2 0 0 0]]
[0 1 2 3]
[108.8 124.8 105.6  99.2]
up
Time Step: 3
[[4 2 0 0]
 [2 0 0 2]
 [0 0 0 0]
 [0 0 0 0]]
[0 1 2 3]
[ 86.4 115.2 102.4  86.4]
up
Time Step: 4
[[4 2 0 2]
 [2 0 0 2]
 [0 0 0 0]
 [0 0 0 0]]
[0 1 2 3]
[115.2 105.6 102.4  94.4]
left
Time Step: 5
[[4 4 0 0]
 [4 0 0 0]
 [0 0 0 0]
 [0 2 0 0]]
[0 1 2 3]
[105.6 108.8  84.8 128. ]
down
Time Step: 6
[[0 0 0 0]
 [0 0 0 0]
 [0 4 0 0]
 [8 2 4 0]]
[0 1 2]
[ 96.  96. 112.   0.]
right
Time Step: 7
[[0 0 0 0]
 [2 0 0 0]
 [0 0 0 4]
 [0 8 2 4]]
[0 1 2 3]
[102.4  96.   96.   86.4]
left
Time Step: 8
[[0 0 0 0]
 [2 0 0 0]
 [4 0 0 0]
 [8 2 4 4]]
[0 1 2]
[144.   92.8  99.2   0. ]
left
Time Step: 9
[[2 0 0 0]
 [2 0 0 0]
 [4 0 0 0]
 [8 2 8 0]]
[1 2 3]
[  0.  102.4 118.4 108.8]
right
Time Step: 10
[[0 0 0 2

In [ ]:
env.render()

In [ ]:
# Epsilon Greedy Algorithm

env = Env2048()
env.reset()

done = False
i = 0
eps = 0.1

while not done:
    print("Time Step: {}".format(i))
    env.render()
    print(env.get_valid_moves())

    move_value = np.zeros(4)

    for move in env.get_valid_moves():
        test_env = env.clone()
        observation, reward, done, info = test_env.step(move)
        for run in range(1,11):
            move_value[move] += (test_env.random_play_simulation_run() - move_value[move])/run
    
    print(move_value)

    if np.random.uniform() < eps:
      action = env.get_random_valid_move()
    else:
      action = np.argmax(move_value)
    
    print(env.action_map[action])
    observation, reward, done, info = env.step(action)
    i += 1
    if done:
        print(env.get_highest_tile())
        break  
    
env.close()

Time Step: 0
[[0 0 0 0]
 [0 0 2 0]
 [0 0 0 2]
 [0 0 0 0]]
[0 1 2 3]
[102.1 118.2  89.3 103.9]
up
Time Step: 1
[[0 2 2 2]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[0 2 3]
[ 94.9   0.  102.3  85.6]
right
Time Step: 2
[[2 0 2 4]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[0 2 3]
[105.2   0.   96.9  90.2]
left
Time Step: 3
[[4 4 0 0]
 [0 0 0 0]
 [0 0 2 0]
 [0 0 0 0]]
[0 1 2 3]
[107.2  94.9 102.7  92.3]
left
Time Step: 4
[[8 0 0 0]
 [0 0 0 0]
 [2 0 2 0]
 [0 0 0 0]]
[0 1 2 3]
[136.6 102.5  84.4 102.1]
left
Time Step: 5
[[8 0 0 0]
 [2 0 0 0]
 [4 0 0 0]
 [0 0 0 0]]
[2 3]
[  0.    0.  117.9  99. ]
right
Time Step: 6
[[0 0 0 8]
 [0 0 0 2]
 [0 0 0 4]
 [0 0 2 0]]
[0 1 2 3]
[113.1  95.  102.6  85. ]
left
Time Step: 7
[[8 0 0 2]
 [2 0 0 0]
 [4 0 0 0]
 [2 0 0 0]]
[0 2 3]
[ 95.3   0.   97.5 101.2]
down
Time Step: 8
[[8 0 0 0]
 [2 0 0 0]
 [4 0 0 2]
 [2 0 0 2]]
[0 1 2 3]
[97.2 97.5 92.4 97. ]
up
Time Step: 9
[[8 0 0 4]
 [2 0 0 0]
 [4 2 0 0]
 [2 0 0 0]]
[0 1 2 3]
[115.7 108.3 103.2  88.6]
left
Time Step: 10
[[8 4 0 0]
 

In [ ]:
env.render()